In [1]:
import json

from promptify.models.nlp.openai_model import OpenAI
from promptify.prompts.nlp.prompter import Prompter

In [2]:
model = OpenAI(api_key="YOUR_API_KEY")
nlp_prompter = Prompter(model)

In [3]:
data = json.load(open("data/tabular4.json",'r'))


In [4]:
len(data['samples'])

3

In [5]:
examples = []
for sample in data['samples'][:5]:
    print(sample,"\n")
    examples.append((sample['Text'],sample['Tabular Data']))

{'Text': 'Deprem sırasında evimizde yer alan adresimiz: İstanbul, Beşiktaş, Yıldız Mahallesi, Cumhuriyet Caddesi No: 35', 'Tabular Data': {'il': 'İstanbul', 'ilçe': 'Beşiktaş', 'mahalle': 'Yıldız Mahallesi', 'sokak': 'Cumhuriyet Caddesi', 'no': 35}} 

{'Text': 'Deprem anında bulunduğumuz adres: Ankara, Kızılay, Mustafa Kemal Paşa Caddesi No: 12/2', 'Tabular Data': {'il': 'Ankara', 'ilçe': 'Kızılay', 'sokak': 'Mustafa Kemal Paşa Caddesi', 'no': '12/2'}} 

{'Text': 'Acil yardıma ihtiyacımız var! Deprem anında bulunduğumuz adres: İzmir, Konak, Alsancak, Cumhuriyet Caddesi No: 45', 'Tabular Data': {'il': 'İzmir', 'ilçe': 'Konak', 'semt': 'Alsancak', 'sokak': 'Cumhuriyet Caddesi', 'no': 45}} 



In [7]:
prompt = nlp_prompter.generate_prompt('geolooc_extractor.jinja',
                                      examples=examples,
                                    #   text_input=data['samples'][6]["Text"],
                                        text_input="Acil yardima ihtiyacim var, Elaziğin merkezindeyim, adresim 1. Sokak 2. Cadde 3. Ev No: 4",

                                     description="Tabular Data Extraction")
print(prompt)

Tabular Data Extraction
You are a highly intelligent and accurate tabular data extractor from plain text input and especially from emergency text that carries address information, your inputs can be text of arbitrary size, but the output should be in [{'tabular': {'entity_type': 'entity'} }] JSON format
Examples:


Input: Deprem sırasında evimizde yer alan adresimiz: İstanbul, Beşiktaş, Yıldız Mahallesi, Cumhuriyet Caddesi No: 35
Output: [{'Tabular': '{'il': 'İstanbul', 'ilçe': 'Beşiktaş', 'mahalle': 'Yıldız Mahallesi', 'sokak': 'Cumhuriyet Caddesi', 'no': 35}' }]

Input: Deprem anında bulunduğumuz adres: Ankara, Kızılay, Mustafa Kemal Paşa Caddesi No: 12/2
Output: [{'Tabular': '{'il': 'Ankara', 'ilçe': 'Kızılay', 'sokak': 'Mustafa Kemal Paşa Caddesi', 'no': '12/2'}' }]

Input: Acil yardıma ihtiyacımız var! Deprem anında bulunduğumuz adres: İzmir, Konak, Alsancak, Cumhuriyet Caddesi No: 45
Output: [{'Tabular': '{'il': 'İzmir', 'ilçe': 'Konak', 'semt': 'Alsancak', 'sokak': 'Cumhuriyet C

In [8]:
output = nlp_prompter.fit('tabular_extractor.jinja',
                                      examples=examples,
                                    #   text_input=data['samples'][6]["Text"],
                                        text_input="Acil yardima ihtiyacim var, Elaziğin merkezindeyim, adresim 1. Sokak 2. Cadde 3. Ev No: 4",
                                      model_name="text-davinci-003"
                                     )

In [9]:
data = output['text'].replace("'{", "{").replace("}'", "}")
eval(data)


[{'Tabular': {'il': 'Elaziğ',
   'sokak': '1. Sokak',
   'cadde': '2. Cadde',
   'ev': '3. Ev',
   'no': 4}}]